# Installation

In [1]:
pip install torch torchvision torchaudio

# Import

In [2]:
import torch

In [3]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

# Neural Network

In [4]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [5]:
model = NeuralNetwork(50, 3)

In [6]:
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [7]:
num_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [8]:
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.1272,  0.0004,  0.0758,  ..., -0.0067, -0.1025, -0.0885],
        [ 0.0931, -0.1360, -0.0108,  ...,  0.1184,  0.0626,  0.1188],
        [-0.1046,  0.0813, -0.1107,  ..., -0.0638,  0.0429, -0.0011],
        ...,
        [-0.0639,  0.0158,  0.0822,  ...,  0.0926,  0.0675, -0.0619],
        [ 0.0429,  0.1060,  0.0594,  ..., -0.0389,  0.0874, -0.0536],
        [-0.1343,  0.0068, -0.1412,  ..., -0.1228,  0.0435,  0.0155]],
       requires_grad=True)


In [9]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [10]:
torch.manual_seed(123)

model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [11]:
torch.manual_seed(123)

X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [12]:
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [13]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


# DataLoader

In [14]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

In [15]:
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [16]:
from torch.utils.data import Dataset


class ToyDataset(Dataset):
  def __init__(self, X, y):
    self.features = X
    self.labels = y

  def __getitem__(self, index):
    one_x = self.features[index]
    one_y = self.labels[index]
    return one_x, one_y

  def __len__(self):
    return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [17]:
len(train_ds)

5

In [18]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0
)

In [19]:
test_ds = ToyDataset(X_test, y_test)

test_loader = DataLoader(
    dataset = test_ds,
    batch_size = 2,
    shuffle = False,
    num_workers = 0
)

In [20]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [21]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [22]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


# A typical training loop

In [23]:
import torch.nn.functional as F


torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):

        # print("Features:", features)
        # print("Labels:", labels)

        logits = model(features)

        loss = F.cross_entropy(logits, labels) # Loss function

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")

    model.eval()
    # Optional model evaluation

Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00


In [24]:
# model.eval()

with torch.no_grad():
    outputs = model(X_train)

print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [25]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[0.9991, 0.0009],
        [0.9982, 0.0018],
        [0.9949, 0.0051],
        [0.0491, 0.9509],
        [0.0307, 0.9693]])


In [26]:
predictions = torch.argmax(probas, dim=1)
print(predictions)


tensor([0, 0, 0, 1, 1])


In [27]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [28]:
predictions == y_train

tensor([True, True, True, True, True])

In [29]:
torch.sum(predictions == y_train)

tensor(5)

In [30]:
def compute_accuracy(model, dataloader):

    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):

        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)

    return (correct / total_examples).item()

In [31]:
compute_accuracy(model, train_loader)

1.0

In [32]:
compute_accuracy(model, test_loader)


1.0

# Saving and loading model

In [33]:
torch.save(model.state_dict(), "model.pth")

In [34]:
model = NeuralNetwork(2, 2) # needs to match the original model exactly
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [35]:
print(torch.cuda.is_available())

True


# GPU optimization

Change runtime to GPU

In [36]:
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])

print(tensor_1 + tensor_2)

tensor([5., 7., 9.])


In [37]:
tensor_1 = tensor_1.to("cuda")
tensor_2 = tensor_2.to("cuda")

print(tensor_1 + tensor_2)

tensor([5., 7., 9.], device='cuda:0')


In [38]:
# tensor_1 = tensor_1.to("cpu")
# print(tensor_1 + tensor_2)

In [39]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

# New: Define a device variable that defaults to a GPU.
device = torch.device("cuda")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # best practice
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") # for mac
# New: Transfer the model onto the GPU.
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):

        # New: Transfer the data onto the GPU.
        features, labels = features.to(device), labels.to(device)    #C
        logits = model(features)
        loss = F.cross_entropy(logits, labels) # Loss function

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")

    model.eval()
    # Optional model evaluation

Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00


# Muli GPU

This will not work on jupyter notebook. Should be run as script and also need mutliple GPUs.

In [40]:
# import torch
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader

# # NEW imports:
# import os
# import platform
# from torch.utils.data.distributed import DistributedSampler
# from torch.nn.parallel import DistributedDataParallel as DDP
# from torch.distributed import init_process_group, destroy_process_group


# # NEW: function to initialize a distributed process group (1 process / GPU)
# # this allows communication among processes
# def ddp_setup(rank, world_size):
#     """
#     Arguments:
#         rank: a unique process ID
#         world_size: total number of processes in the group
#     """
#     # Only set MASTER_ADDR and MASTER_PORT if not already defined by torchrun
#     if "MASTER_ADDR" not in os.environ:
#         os.environ["MASTER_ADDR"] = "localhost"
#     if "MASTER_PORT" not in os.environ:
#         os.environ["MASTER_PORT"] = "12345"

#     # initialize process group
#     if platform.system() == "Windows":
#         # Disable libuv because PyTorch for Windows isn't built with support
#         os.environ["USE_LIBUV"] = "0"
#         # Windows users may have to use "gloo" instead of "nccl" as backend
#         # gloo: Facebook Collective Communication Library
#         init_process_group(backend="gloo", rank=rank, world_size=world_size)
#     else:
#         # nccl: NVIDIA Collective Communication Library
#         init_process_group(backend="nccl", rank=rank, world_size=world_size)

#     torch.cuda.set_device(rank)


# class ToyDataset(Dataset):
#     def __init__(self, X, y):
#         self.features = X
#         self.labels = y

#     def __getitem__(self, index):
#         one_x = self.features[index]
#         one_y = self.labels[index]
#         return one_x, one_y

#     def __len__(self):
#         return self.labels.shape[0]


# class NeuralNetwork(torch.nn.Module):
#     def __init__(self, num_inputs, num_outputs):
#         super().__init__()

#         self.layers = torch.nn.Sequential(
#             # 1st hidden layer
#             torch.nn.Linear(num_inputs, 30),
#             torch.nn.ReLU(),

#             # 2nd hidden layer
#             torch.nn.Linear(30, 20),
#             torch.nn.ReLU(),

#             # output layer
#             torch.nn.Linear(20, num_outputs),
#         )

#     def forward(self, x):
#         logits = self.layers(x)
#         return logits


# def prepare_dataset():
#     X_train = torch.tensor([
#         [-1.2, 3.1],
#         [-0.9, 2.9],
#         [-0.5, 2.6],
#         [2.3, -1.1],
#         [2.7, -1.5]
#     ])
#     y_train = torch.tensor([0, 0, 0, 1, 1])

#     X_test = torch.tensor([
#         [-0.8, 2.8],
#         [2.6, -1.6],
#     ])
#     y_test = torch.tensor([0, 1])

#     # Uncomment these lines to increase the dataset size to run this script on up to 8 GPUs:
#     # factor = 4
#     # X_train = torch.cat([X_train + torch.randn_like(X_train) * 0.1 for _ in range(factor)])
#     # y_train = y_train.repeat(factor)
#     # X_test = torch.cat([X_test + torch.randn_like(X_test) * 0.1 for _ in range(factor)])
#     # y_test = y_test.repeat(factor)

#     train_ds = ToyDataset(X_train, y_train)
#     test_ds = ToyDataset(X_test, y_test)

#     train_loader = DataLoader(
#         dataset=train_ds,
#         batch_size=2,
#         shuffle=False,  # NEW: False because of DistributedSampler below
#         pin_memory=True,
#         drop_last=True,
#         # NEW: chunk batches across GPUs without overlapping samples:
#         sampler=DistributedSampler(train_ds)  # NEW
#     )
#     test_loader = DataLoader(
#         dataset=test_ds,
#         batch_size=2,
#         shuffle=False,
#     )
#     return train_loader, test_loader


# # NEW: wrapper
# def main(rank, world_size, num_epochs):

#     ddp_setup(rank, world_size)  # NEW: initialize process groups

#     train_loader, test_loader = prepare_dataset()
#     model = NeuralNetwork(num_inputs=2, num_outputs=2)
#     model.to(rank)
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

#     model = DDP(model, device_ids=[rank])  # NEW: wrap model with DDP
#     # the core model is now accessible as model.module

#     for epoch in range(num_epochs):
#         # NEW: Set sampler to ensure each epoch has a different shuffle order
#         train_loader.sampler.set_epoch(epoch)

#         model.train()
#         for features, labels in train_loader:

#             features, labels = features.to(rank), labels.to(rank)  # New: use rank
#             logits = model(features)
#             loss = F.cross_entropy(logits, labels)  # Loss function

#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             # LOGGING
#             print(f"[GPU{rank}] Epoch: {epoch+1:03d}/{num_epochs:03d}"
#                   f" | Batchsize {labels.shape[0]:03d}"
#                   f" | Train/Val Loss: {loss:.2f}")

#     model.eval()

#     try:
#         train_acc = compute_accuracy(model, train_loader, device=rank)
#         print(f"[GPU{rank}] Training accuracy", train_acc)
#         test_acc = compute_accuracy(model, test_loader, device=rank)
#         print(f"[GPU{rank}] Test accuracy", test_acc)

#     ####################################################
#     # NEW:
#     except ZeroDivisionError as e:
#         raise ZeroDivisionError(
#             f"{e}\n\nThis script is designed for 2 GPUs. You can run it as:\n"
#             "torchrun --nproc_per_node=2 DDP-script-torchrun.py\n"
#             f"Or, to run it on {torch.cuda.device_count()} GPUs, uncomment the code on lines 103 to 107."
#         )
#     ####################################################

#     destroy_process_group()  # NEW: cleanly exit distributed mode


# def compute_accuracy(model, dataloader, device):
#     model = model.eval()
#     correct = 0.0
#     total_examples = 0

#     for idx, (features, labels) in enumerate(dataloader):
#         features, labels = features.to(device), labels.to(device)

#         with torch.no_grad():
#             logits = model(features)
#         predictions = torch.argmax(logits, dim=1)
#         compare = labels == predictions
#         correct += torch.sum(compare)
#         total_examples += len(compare)
#     return (correct / total_examples).item()


# if __name__ == "__main__":
#     # NEW: Use environment variables set by torchrun if available, otherwise default to single-process.
#     if "WORLD_SIZE" in os.environ:
#         world_size = int(os.environ["WORLD_SIZE"])
#     else:
#         world_size = 1

#     if "LOCAL_RANK" in os.environ:
#         rank = int(os.environ["LOCAL_RANK"])
#     elif "RANK" in os.environ:
#         rank = int(os.environ["RANK"])
#     else:
#         rank = 0

#     # Only print on rank 0 to avoid duplicate prints from each GPU process
#     if rank == 0:
#         print("PyTorch version:", torch.__version__)
#         print("CUDA available:", torch.cuda.is_available())
#         print("Number of GPUs available:", torch.cuda.device_count())

#     torch.manual_seed(123)
#     num_epochs = 3
#     main(rank, world_size, num_epochs)